In [1]:
#Requirements
!pip install folium
!pip install mapclassify

In [2]:
!pip install pycountry

In [3]:
import geopandas as gpd
import pandas as pd
import pycountry
import numpy as np

In [4]:
world = gpd.read_file(gpd.datasets.get_path('naturalearth_lowres'))


In [5]:
df_taux_fertilité=pd.read_csv('data_natalité.csv')
df_taux_fertilité.rename(columns={'Fertility Rate   In 2019 (Births/Woman)': 'Fertility'}, inplace=True)
df_taux_fertilité.head(5)

,Country Name,Fertility
0,Niger,6.824
1,Somalia,5.978
2,DR Congo,5.819
3,Mali,5.785
4,Chad,5.649


In [6]:
#Fonction pour donnner le nom de code du pays des pays scrappés pour le taux de fertilité 
def code_pays(column):
    CODE=[]
    for country in column:
        try:
            code=pycountry.countries.get(name=country).alpha_3
           # .alpha_3 est utilisé pour donner le code en 3 lettres du pays
            CODE.append(code)
        except:
            CODE.append(np.NaN)
    return CODE# create a column for code 

df_taux_fertilité['CODE']=code_pays(df_taux_fertilité["Country Name"])

In [7]:
df_taux_fertilité.rename(columns={'CODE': 'Country Code'}, inplace=True)

In [8]:
df_famille=df_taux_fertilité

In [9]:
def shorten_float(nb: float)-> float:
    return int(nb*100)/100.

In [10]:
df_famille['Fertility'].astype

<bound method NDFrame.astype of 0           6.824
1           5.978
2           5.819
3           5.785
4           5.649
          ...    
259    2.1 (2002)
260             -
261             -
262             -
263             -
Name: Fertility, Length: 264, dtype: object>

In [11]:
def fertility_column_cleaner(char: str ):
    try:
        if '-' in char : return np.NaN
        elif '(' in char :
            i=0
            while char[i+1]!='(':
                i+=1
            return float(char[:i])
        else: return float(char)
    
    except:
        return char
    

In [12]:
df_famille["Fertility"] = df_famille["Fertility"].apply(fertility_column_cleaner)
df_famille.dropna(inplace=True)

On voit que la régression marche bien ce qui justifie l'emploie de cet indicateur.  

In [13]:
df_famille2=df_famille

In [14]:
df_famille2=df_famille2[["Country Code", "Fertility"]]
df_famille2

,Country Code,Fertility
0,NER,6.824
1,SOM,5.978
3,MLI,5.785
4,TCD,5.649
5,AGO,5.442
...,...,...
246,DMA,1.900
251,MHL,4.000
256,PLW,2.200
257,SMR,1.300


En utilisant la technique avec le Code on obtient 159 valeurs au total contre 171 quand on faisait avec Country Name donc il y a potentiellement des mismatch --> problématique ?

In [15]:
def normalize_columns(df: pd.DataFrame, columns: list)-> pd.DataFrame:
    
    for col in columns:
        try:

            df.loc[:,col] = (df.loc[:,col]- df.loc[:,col].min())/(df.loc[:,col].max()- df.loc[:,col].min())
        except:
            print(col + " n'a pas été normalisé \n")
    return df
            


In [16]:
df_famille2 = normalize_columns(df_famille2,["Fertility"])
df_famille2['Fertility']=df_famille2['Fertility'].apply(shorten_float)

C:\Users\Ca1000\anaconda3\envs\projetPython2A\lib\site-packages\pandas\core\indexing.py:1773: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(ilocs[0], value, pi)
C:\Users\Ca1000\AppData\Local\Temp/ipykernel_18480/2181270134.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_famille2['Fertility']=df_famille2['Fertility'].apply(shorten_float)


In [17]:
df_famille2.set_index(["Country Code"], inplace=True)

In [18]:
df_famille2

,Fertility
Country Code,
NER,1.00
SOM,0.85
MLI,0.81
TCD,0.79
AGO,0.75
...,...
DMA,0.13
MHL,0.50
PLW,0.19


In [19]:
world.merge(df_famille2, how = 'inner' ,right_on = 'Country Code', left_on = 'iso_a3')

,pop_est,continent,name,iso_a3,gdp_md_est,geometry,Fertility
0,920938,Oceania,Fiji,FJI,8374.0,"MULTIPOLYGON (((180.00000 -16.06713, 180.00000...",0.28
1,35623680,North America,Canada,CAN,1674000.0,"MULTIPOLYGON (((-122.84000 49.00000, -122.9742...",0.06
2,326625791,North America,United States of America,USA,18560000.0,"MULTIPOLYGON (((-122.84000 49.00000, -120.0000...",0.10
3,18556698,Asia,Kazakhstan,KAZ,460700.0,"POLYGON ((87.35997 49.21498, 86.59878 48.54918...",0.31
4,29748859,Asia,Uzbekistan,UZB,202300.0,"POLYGON ((55.96819 41.30864, 55.92892 44.99586...",0.29
...,...,...,...,...,...,...,...
140,2103721,Europe,Macedonia,MKD,29520.0,"POLYGON ((22.38053 42.32026, 22.88137 41.99930...",0.06
141,7111024,Europe,Serbia,SRB,101800.0,"POLYGON ((18.82982 45.90887, 18.82984 45.90888...",0.07
142,642550,Europe,Montenegro,MNE,10610.0,"POLYGON ((20.07070 42.58863, 19.80161 42.50009...",0.11
143,1218208,North America,Trinidad and Tobago,TTO,43570.0,"POLYGON ((-61.68000 10.76000, -61.10500 10.890...",0.10


In [20]:
world

,pop_est,continent,name,iso_a3,gdp_md_est,geometry
0,920938,Oceania,Fiji,FJI,8374.0,"MULTIPOLYGON (((180.00000 -16.06713, 180.00000..."
1,53950935,Africa,Tanzania,TZA,150600.0,"POLYGON ((33.90371 -0.95000, 34.07262 -1.05982..."
2,603253,Africa,W. Sahara,ESH,906.5,"POLYGON ((-8.66559 27.65643, -8.66512 27.58948..."
3,35623680,North America,Canada,CAN,1674000.0,"MULTIPOLYGON (((-122.84000 49.00000, -122.9742..."
4,326625791,North America,United States of America,USA,18560000.0,"MULTIPOLYGON (((-122.84000 49.00000, -120.0000..."
...,...,...,...,...,...,...
172,7111024,Europe,Serbia,SRB,101800.0,"POLYGON ((18.82982 45.90887, 18.82984 45.90888..."
173,642550,Europe,Montenegro,MNE,10610.0,"POLYGON ((20.07070 42.58863, 19.80161 42.50009..."
174,1895250,Europe,Kosovo,-99,18490.0,"POLYGON ((20.59025 41.85541, 20.52295 42.21787..."
175,1218208,North America,Trinidad and Tobago,TTO,43570.0,"POLYGON ((-61.68000 10.76000, -61.10500 10.890..."


In [22]:
world.merge(df_famille2, how = 'inner' ,right_on = 'Country Code', left_on = 'iso_a3').explore(
     column="Fertility", # make choropleth based on "BoroName" column
     tooltip="Fertility", # show "BoroName" value in tooltip (on hover)
     popup=True, # show all values in popup (on click)
     tiles="CartoDB positron", # use "CartoDB positron" tiles
     cmap="Set1", # use "Set1" matplotlib colormap
     style_kwds=dict(color="black") # use black outline
    )

AttributeError: 'GeoDataFrame' object has no attribute 'explore'